<a href="https://colab.research.google.com/github/suelen-prs/-ebac-ciencia-de-dados-modulo-3-parte-2/blob/master/Mod13_Tarefa02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [9]:
import graphviz

import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [41]:
df = pd.read_csv('previsao_de_renda.csv')

In [42]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   index                  15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12466 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  mau                    15000 non-null  bool   
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [44]:
df.head(20)

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,mau,renda
0,2015-01-01,8682,F,False,True,1,Assalariado,Secundário,Casado,Casa,36,3.575342,3.0,False,3369.24
1,2015-01-01,12830,M,True,True,0,Empresário,Secundário,Casado,Casa,42,0.860274,2.0,False,6096.14
2,2015-01-01,2885,M,True,True,2,Assalariado,Superior completo,Casado,Casa,31,8.065753,4.0,False,5658.98
3,2015-01-01,16168,F,True,False,0,Empresário,Secundário,Casado,Casa,50,1.208219,2.0,False,7246.69
4,2015-01-01,12438,M,False,False,0,Assalariado,Secundário,Casado,Casa,52,13.873973,2.0,False,4017.37
5,2015-01-01,7409,M,False,True,0,Assalariado,Superior incompleto,Casado,Casa,46,15.561644,2.0,False,5483.58
6,2015-01-01,6913,F,False,True,0,Assalariado,Secundário,Casado,Casa,51,5.046575,2.0,False,1947.99
7,2015-01-01,13047,M,True,True,0,Empresário,Secundário,Casado,Casa,24,1.591781,2.0,False,7781.51
8,2015-01-01,4529,F,False,False,0,Assalariado,Secundário,Casado,Casa,53,3.430137,2.0,False,5577.82
9,2015-01-01,14646,F,False,True,2,Servidor público,Secundário,União,Casa,34,7.219178,4.0,False,803.34


# 1.Separe a base em treinamento e teste (25% para teste, 75% para treinamento).


In [45]:
# Separando o target e as features
y_train = np.log(df["renda"])
X_train = df.drop("renda", axis=1)
y_test = np.log(df["renda"])
X_test = df.drop("renda", axis=1)

In [46]:
# Ajustando o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', drop='first')
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

# 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [47]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_encoded, y_train)
    predictions = ridge.predict(X_test_encoded)
    r2 = r2_score(y_test, predictions)
    ridge_results[alpha] = r2
    print(f'Ride: alpha = {alpha}, r2 = {r2}')

best_alpha_ridge = max(ridge_results, key=ridge_results.get)
print(f"Melhor alpha (Ridge): {best_alpha_ridge} com R^2 = {ridge_results[best_alpha_ridge]}")


Ride: alpha = 0, r2 = 0.9999996527895966
Ride: alpha = 0.001, r2 = 0.9999991040687036
Ride: alpha = 0.005, r2 = 0.9999915596763116
Ride: alpha = 0.01, r2 = 0.9999682519628522
Ride: alpha = 0.05, r2 = 0.9992697626754319
Ride: alpha = 0.1, r2 = 0.9973067902875641
Melhor alpha (Ridge): 0 com R^2 = 0.9999996527895966


O modelo que possui apenas dados estatisticamente relevantes e com menor Alpha é o 0.01, aumentando o Alpha acabamos perdendo variáveis relevantes ao modelo, portanto acredito que o melhor modelo seja o de Alpha = 0.01.

# 3.Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?


In [48]:
lasso_results = {}

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_encoded, y_train)
    predictions = lasso.predict(X_test_encoded)
    r2 = r2_score(y_test, predictions)
    lasso_results[alpha] = r2
    print(f'Lasso: r2 = {r2}')

best_alpha_lasso = max(lasso_results, key=lasso_results.get)
print(f"Melhor alpha (LASSO): {best_alpha_lasso} com R^2 = {lasso_results[best_alpha_lasso]}")

# Comparação
if ridge_results[best_alpha_ridge] > lasso_results[best_alpha_lasso]:
    print("Ridge tem um melhor R^2")
else:
    print("LASSO tem um melhor R^2")


<ipython-input-48-2ca5eaf092f0>:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train_encoded, y_train)


Lasso: r2 = 0.9999997482097542
Lasso: r2 = 0.10872424239709133
Lasso: r2 = 0.08293934697321426
Lasso: r2 = 0.07371340868577747
Lasso: r2 = 0.037630734733224425
Lasso: r2 = 0.0
Melhor alpha (LASSO): 0 com R^2 = 0.9999997482097542
LASSO tem um melhor R^2


# 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [49]:
df1 = df.copy()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   index                  15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12466 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  mau                    15000 non-null  bool   
 14  renda                  15000 non-null  float64
dtypes:

In [50]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 1. Remover a coluna data_ref
df1 = df1.drop('data_ref', axis=1)

# 2. Converter colunas categóricas para representações numéricas
df1 = pd.get_dummies(df1, drop_first=True)

# Converta colunas booleanas para int
for col in ['posse_de_veiculo', 'posse_de_imovel', 'mau']:
    df1[col] = df1[col].astype(int)

# Tratar NaNs
df1.fillna(df1.mean(), inplace=True)

# Dividir em treino e teste
X = df1.drop('renda', axis=1)
y = df1['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Seleção de variáveis stepwise (backward)
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const).fit()

while True:
    p_values = model.pvalues.drop('const')
    max_p_value = p_values.max()
    if max_p_value > 0.05:
        drop_var = p_values.idxmax()
        X_train_const = X_train_const.drop(drop_var, axis=1)
        model = sm.OLS(y_train, X_train_const).fit()
    else:
        break

# Avaliação no conjunto de teste
X_test_const = sm.add_constant(X_test[X_train_const.columns.drop('const')])
y_pred = model.predict(X_test_const)
r2 = r2_score(y_test, y_pred)
print(f"R^2 no conjunto de teste: {r2:.2f}")


R^2 no conjunto de teste: 0.24


# 5.Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

As três regressões acabam sendo boas, excluiria o modelo da Regressão de Ridge devido a quantidade de variáveis explicativas, e escolho a LASSO como melhor de todos.

# 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.


In [59]:
# Transformação logarítmica
df1['log_renda'] = np.log(df1['renda'])

In [60]:
# Interação entre variáveis
df1['idade_tempo_emprego'] = df1['idade'] * df1['tempo_emprego']


In [61]:
# Variável polinomial
df1['idade_quadrado'] = df1['idade'] ** 2


In [62]:
# Separando o target e as features
y1_train = np.log(df1["renda"])
X1_train = df1.drop("renda", axis=1)
y1_test = np.log(df1["renda"])
X1_test = df1.drop("renda", axis=1)

In [63]:
# Ajustando o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', drop='first')
X_train_encoded1 = encoder.fit_transform(X1_train)
X_test_encoded1 = encoder.transform(X1_test)

In [64]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_encoded1, y1_train)
    predictions = ridge.predict(X_test_encoded1)
    r2 = r2_score(y1_test, predictions)
    ridge_results[alpha] = r2
    print(f'Ride: alpha = {alpha}, r2 = {r2}')

best_alpha_ridge = max(ridge_results, key=ridge_results.get)
print(f"Melhor alpha (Ridge): {best_alpha_ridge} com R^2 = {ridge_results[best_alpha_ridge]}")

Ride: alpha = 0, r2 = 0.9999999947161793
Ride: alpha = 0.001, r2 = 0.9999999256820972
Ride: alpha = 0.005, r2 = 0.9999982757762934
Ride: alpha = 0.01, r2 = 0.9999931501007896
Ride: alpha = 0.05, r2 = 0.9998343521127169
Ride: alpha = 0.1, r2 = 0.9993630942805221
Melhor alpha (Ridge): 0 com R^2 = 0.9999999947161793


# 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [67]:
from sklearn.tree import DecisionTreeRegressor

tree_regressor = DecisionTreeRegressor(max_depth=5)  # Você pode ajustar a profundidade máxima como quiser
tree_regressor.fit(X1_train, y1_train)

y_pred_tree = tree_regressor.predict(X1_test)
r2_tree = r2_score(y1_test, y_pred_tree)

print(f"R^2 para Árvore de Regressão: {r2_tree}")


R^2 para Árvore de Regressão: 0.9974303425136081
